In [4]:
# Statistical Analysis
# Programming: Selime Gurol (CERFACS), 2021

import numpy as np
from numpy import zeros, eye, exp
from numpy.linalg import \
    (inv,# To invert a matrix
    norm,# To compute the Euclidean norm
    cholesky) # To compute Cholesky factorization
from numpy.random import randn # To generate samples from a normalized Gaussian
import matplotlib.pyplot as plt # To plot a grapH


In [31]:

def statistical_analysis(sigma0=0.4, locations=(3,), btype='diagonal', sigmaB=0.8, correlation_length=1.0, plot=True):
    ##%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    #                          (1) Initialization
    ##%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    n  = 4  # state space dimension 

    # Observation operator
    I = eye(n)
    inds_obs = list(locations) # [3] # Location of the observations (1 dimensional grid)
    H = I[inds_obs] # H as a selection operator - H is size m x n
    m = len(inds_obs) # number of observations

    # Observation errors
    # R is a diagonal matrix
    sigmaR = sigma0 # 0.4 # observation error std
    R = zeros((m,m))
    for ii in range(m):
        R[ii,ii] = sigmaR*sigmaR 

    # Background errors
    sigmaB = sigmaB # 0.8 # background error std
    L = correlation_length # 1.0 # correlation length scale
    btype = btype #'diagonal'
    B = zeros((n,n))
    if btype == 'diagonal':
        # no correlation between the grid points
        for ii in range(n):
            B[ii,ii] = sigmaB*sigmaB  
    if btype == 'soar':
        # correlation between the grid points
        for ii in range(n):
            for jj in range(n):
                rij = abs(jj-ii)
                rho = (1 + rij/L)*exp(-rij/L)
                B[ii,jj] = sigmaB*sigmaB*rho          
    # B = B12 * B12^T
    B12 = cholesky(B)
    ##%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    #                          (2) Generate the truth
    ##%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    xt = randn(n) # the true state

    ##%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    #                          (3) Generate the background
    ##%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    xb = xt + np.einsum('ij,jm->im', B12, randn(n, m))

    ##%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    #                          (4) Generate the observations
    ##%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    y = np.einsum('mn, n-> m', H, xt) + np.sqrt(R) * randn(1, m) # as long as R is diagonal

    ##%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    #                          (5) Obtain analysis from BLUE
    ##%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    #Kalman Gain matrix
    BGt = np.matmul(B, H.T)
    K = np.matmul(BGt, inv(np.matmul(H, BGt) + R)) # TO DO
    #BLUE analysis
    xa = xb + K * (y - H * xb)

    ##%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    #                          (5) Diagnostics
    ##%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    print('')
    relative_err_xb = norm(xt - xb)/norm(xt)
    relative_err_xa = norm(xt - xa)/norm(xt)
    if(plot):
        print('')
        print('||xt - xb||_2 / ||xt||_2 = ', relative_err_xb)
        print('||xt - xa||_2 / ||xt||_2 = ', relative_err_xa)
        print('\n')

    # Analysis covariance matrix
    Sinv = inv(B) + (H.T).dot(inv(R).dot(H))
    S = inv(Sinv)
    if(plot):
        print('Analysis covariance matrix: \n')
    trace = []
    for ii in range(n):
        if(plot):
            print('S[{}, {}]: {}'.format(ii, ii, S[ii,ii]))
        trace.append(S[ii, ii])

    ##%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    #                          (6) Plots
    ##%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    if(plot):
        xrange = range(0,n)
        fig, ax = plt.subplots()
        ax.plot(xrange, xt, '+k', label='Truth')
        ax.plot(xrange, xb, 'db', label='Background')
        ax.plot(inds_obs, y, 'og', label='Observations')
        ax.plot(xrange, xa, 'xr', label='Analysis')
        leg = ax.legend()
        plt.xlabel('x-coordinate')
        plt.ylabel('Temperature')
        plt.show()
    return {
        'relative_err_xb': relative_err_xb,
        'relative_err_xa': relative_err_xa,
        'covariance_mat_trace': trace,
        'xt': xt,
        'xb': xb,
        'y': y,
        'xa': xa,
        'inds_obs': inds_obs
    }

In [33]:
# Q3.1 # soar , diagonal
# Modify : locations, sigma0
# since diagonal, should not change too much things since no background correlation between positions
# having one observation only is useful for the given observation... (i.e. background = analysis)
results_q31a = statistical_analysis(sigma0=0.4, locations=(3,), btype='diagonal', sigmaB=0.8, correlation_length=1.0, plot=False)
results_q31b = statistical_analysis(sigma0=0.4, locations=(1,), btype='diagonal', sigmaB=0.8, correlation_length=1.0, plot=False)
results_q31c = statistical_analysis(sigma0=0.1, locations=(3,), btype='diagonal', sigmaB=0.8, correlation_length=1.0, plot=False)
results_q31d = statistical_analysis(sigma0=0.8, locations=(3,), btype='diagonal', sigmaB=0.8, correlation_length=1.0, plot=False)

In [34]:
# Q3.2
# We now see the observation obtained on one specific grid point being useful for other grid points
results_q32a = statistical_analysis(sigma0=0.4, locations=(3,), btype='diagonal', sigmaB=0.8, correlation_length=1.0, plot=False)
results_q32b = statistical_analysis(sigma0=0.4, locations=(3,), btype='soar', sigmaB=0.8, correlation_length=1.0, plot=False)


In [35]:
# Q3.3
results_q33a = statistical_analysis(sigma0=0.4, locations=(3,), btype='soar', sigmaB=0.8, correlation_length=1.0, plot=False)
results_q33b = statistical_analysis(sigma0=0.4, locations=(3,), btype='soar', sigmaB=0.8, correlation_length=2.0, plot=False)
results_q33c = statistical_analysis(sigma0=0.4, locations=(3,), btype='soar', sigmaB=0.8, correlation_length=0.5, plot=False)
results_q33d = statistical_analysis(sigma0=0.4, locations=(3,), btype='soar', sigmaB=0.2, correlation_length=1.0, plot=False)
